In [1]:
from pathlib import Path
import sys
import json


# Поднимаемся на два уровня вверх (из notebooks/ в InteriorClass/)
project_root = Path.cwd().parent.parent
sys.path.append(str(project_root))  # Теперь Python увидит src/


from tqdm import tqdm
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
import torchvision.transforms as transforms
from PIL import Image

from src.config import RANDOM_SEED, SPLIT_RATIO, MIN_VAL_TEST_PER_CLASS, CLASS_LABELS
from src.dataset.splitter import DatasetSplitter
from src.dataset.interior_dataset import InteriorDataset, get_transforms
from src.models.interior_classifier_EfficientNet_B3 import InteriorClassifier
from src.trainer import Trainer


In [2]:
# 1. Define hyperparameters
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 32
EPOCHS = 10
LR = 3e-5
IMG_SIZE = 380  # Для EfficientNet-B3
EXP_NUMBER = 4

In [3]:
# 2. Define paths
notebook_dir = Path.cwd()
project_root = notebook_dir.parent.parent
data_dir = project_root / "data"
print(f"data_dir: {data_dir}")

dataset_dir = data_dir / "interior_dataset"

samples = InteriorDataset.collect_samples(dataset_dir=dataset_dir)
print(f"Total samples: {len(samples)}")

data_dir: /home/little-garden/CodeProjects/InteriorClass/data


Total samples: 54591


In [4]:
# 3. Create DatasetSplitter
splitter = DatasetSplitter(
    class_labels=CLASS_LABELS,
    split_config=SPLIT_RATIO,
    random_seed=RANDOM_SEED
)


train_samples, val_samples, test_samples = splitter.split(samples, shuffle=True)
print(f"Train samples: {len(train_samples)}")
print(f"Val samples: {len(val_samples)}")
print(f"Test samples: {len(test_samples)}")

# 4. Create Datasets
train_dataset = InteriorDataset(
    train_samples,
    transform=get_transforms(img_size=IMG_SIZE, mode='train'),
    mode='train'
)

val_dataset = InteriorDataset(
    val_samples,
    transform=get_transforms(img_size=IMG_SIZE, mode='val'),
    mode='val'
)

test_dataset = InteriorDataset(
    test_samples,
    transform=get_transforms(img_size=IMG_SIZE, mode='test'),
    mode='test'
)


# 5. Create DataLoaders
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    drop_last=True
)

val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

Train samples: 47649
Val samples: 3464
Test samples: 3478


In [5]:
# 6. Initializing model
model = InteriorClassifier(num_classes=len(CLASS_LABELS)).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=LR)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

experiments_dir = project_root / "experiments"
exp_dir = experiments_dir / f"exp{EXP_NUMBER:03d}"
exp_results_dir = exp_dir / "results"
exp_results_dir.mkdir(parents=True, exist_ok=True)

def load_latest_file(pattern: str) -> Path | None:
    """Находит самый свежий файл по паттерну"""
    files = list(exp_results_dir.glob(pattern))
    if not files:
        return None
    # Сортируем по дате изменения (новейший первый)
    files.sort(key=lambda x: x.stat().st_mtime, reverse=True)
    return files[0]

# Пытаемся загрузить чекпоинт
checkpoint_path = load_latest_file("ckpt*")
if checkpoint_path:
    try:
        checkpoint = torch.load(checkpoint_path, map_location=DEVICE)
        print(f"Loaded checkpoint from: {checkpoint_path.name}")
        
        model.load_state_dict(checkpoint['model_state_dict'])
        if 'optimizer_state_dict' in checkpoint:
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        
        print(f"Successfully loaded checkpoint (epoch {checkpoint.get('epoch', 'unknown')})")
    except Exception as e:
        print(f"Error loading checkpoint {checkpoint_path}: {str(e)}")
        checkpoint_path = None

# Если чекпоинта нет, пробуем загрузить полную модель
if not checkpoint_path:
    model_path = load_latest_file("model*")
    if model_path:
        try:
            # Для полной модели не нужен load_state_dict
            model = torch.load(model_path, map_location=DEVICE)
            print(f"Successfully loaded full model from: {model_path.name}")
        except Exception as e:
            print(f"Error loading model {model_path}: {str(e)}")
            model_path = None

# Если ничего не загрузилось - исключение
if not checkpoint_path and not model_path:
    available_files = [f.name for f in exp_results_dir.iterdir() if f.is_file()]
    raise FileNotFoundError(
        f"No valid checkpoint or model found in {exp_results_dir}\n"
        f"Available files: {available_files or 'None'}"
    )

# 7. Creating Trainer
trainer = Trainer(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    sheduler=scheduler,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    epochs=EPOCHS,
    device=DEVICE,
    exp_results_dir=exp_results_dir

)

Loaded checkpoint from: checkpoint.pth
Successfully loaded checkpoint (epoch 7)


In [6]:
trainer.save_model()

/home/little-garden/CodeProjects/InteriorClass/experiments/exp004/results/model.pth


In [ ]:
test_report = trainer.test()

In [13]:
log_path = exp_results_dir / "training_log.json"

In [14]:
if log_path.exists():
    with open(log_path, "r") as f:
        log_dict = json.load(f)

In [ ]:
test_report

In [17]:
log_dict['test_report'] = test_report

In [18]:
with open(log_path, "w") as f:
    json.dump(log_dict, f, indent=4)

In [ ]:
# Пути для сохранения результатов
exp_dir = root_project / Path("experiments/exp001_baseline/results")
exp_dir.mkdir(parents=True, exist_ok=True)

# Файлы для сохранения
checkpoint_path = exp_dir / "best_model.pth"
log_path = exp_dir / "training_log.json"

# Инициализация лога
if log_path.exists():
    with open(log_path, "r") as f:
        log = json.load(f)
    best_val_loss = log.get("best_val_loss", float("inf"))
else:
    log = {"train_loss": [], "val_loss": [], "val_accuracy": [], "best_val_loss": float("inf")}
    best_val_loss = float("inf")


# Модель и оптимизатор
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = InteriorClassifier(num_classes=len(InteriorDataset.CLASSES)).to(device)

checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['model_state_dict'])

# Финальная оценка на тестовом наборе
model.eval()
test_preds, test_labels = [], []

test_bar = tqdm(
    test_loader, 
    desc='Final Testing',
    bar_format='{l_bar}{bar:20}{r_bar}{bar:-20b}'
)

with torch.no_grad():
    for inputs, labels in test_bar:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        
        test_preds.extend(preds.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

# Компактный финальный отчет
print("\nFinal Test Results:")
final_report = classification_report(
    test_labels, test_preds,
    target_names=InteriorDataset.CLASSES,
    digits=4,
    output_dict=True
)

test_accuracy = final_report['accuracy']
print(f"Test Accuracy: {test_accuracy:.4f}")
print(
    f"Macro Avg: P={final_report['macro avg']['precision']:.4f} "
    f"R={final_report['macro avg']['recall']:.4f} "
    f"F1={final_report['macro avg']['f1-score']:.4f}"
)

# Сохраняем финальные результаты в лог
log["test_accuracy"] = test_accuracy
log["test_report"] = final_report
with open(log_path, "w") as f:
    json.dump(log, f, indent=4)


In [5]:
def prepare_image(image_path, img_size=(224, 224)):
    # Трансформы должны быть такими же, как при обучении!
    transform = transforms.Compose([
        transforms.Resize(img_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    img = Image.open(image_path).convert("RGB")  # Обязательно конвертируем в RGB
    return transform(img).unsqueeze(0)  # Добавляем batch-размерность

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = 'cpu'
model = InteriorClassifier(num_classes=len(InteriorDataset.CLASSES)).to(device)

exp_dir = root_project / Path("experiments/exp001_baseline/results")
checkpoint_path = exp_dir / "best_model.pth"
checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['model_state_dict'])

# Финальная оценка на тестовом наборе
model.eval()
class_idx2image_path = {}
for image_path, class_idx in test_samples:
    if class_idx not in class_idx2image_path:
        class_idx2image_path[class_idx] = image_path

print(*class_idx2image_path.items(), sep='\n', end='\n\n')

# input_tensor = prepare_image(image_path).to(device)

# print(input_tensor.shape)

with torch.no_grad():  # Отключаем вычисление градиентов
    for class_idx, image_path in class_idx2image_path.items():
        image = prepare_image(image_path).to(device)
        outputs = model(image)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        predicted_class = torch.argmax(probabilities).item()
        
        # probabilities = [p for p in probabilities[0]]
        print(f"File: {image_path} | class_idx={class_idx}")
        print(f"Probabilities: {[round(float(p), 4) for p in probabilities[0]]}", end='\n')
        print(f"Predicted class: {predicted_class} ({InteriorDataset.CLASSES[predicted_class]}) | Confidence: {probabilities[0][predicted_class].item():.4f}\n\n")

In [ ]:
B1_path_test_image = dataset_dir / "B1"/ "B1_13070_56d9.jpg"
with torch.no_grad():  # Отключаем вычисление градиентов
    image = prepare_image(B1_path_test_image).to(device)
    outputs = model(image)
    probabilities = torch.nn.functional.softmax(outputs, dim=1)
    predicted_class = torch.argmax(probabilities).item()

    print(f"File: {B1_path_test_image} | class_idx={class_idx}")
    print(f"Probabilities: {[round(float(p), 4) for p in probabilities[0]]}", end='\n')
    print(f"Predicted class: {predicted_class} ({InteriorDataset.CLASSES[predicted_class]}) | Confidence: {probabilities[0][predicted_class].item():.4f}\n\n")

In [ ]:
for p, idx in test_samples:
    if idx == 3:
        print(p, idx)